In [1]:
import pandas as pd
import nltk
import textgrid
import os
import random
import spacy
import torch
root_dir = './'
word_level_timing = root_dir+'word_level_timing'
motion_label = root_dir+'motion_labels'

In [2]:
german_text = "Die Sonne scheint am Himmel und die Vögel singen."
german_tagger = spacy.load("de_core_news_sm")
doc = german_tagger(german_text)
print(doc)
# Extract tokens and POS tags
# for token in doc:
#     print(token.text, token.pos_)

Die Sonne scheint am Himmel und die Vögel singen.


In [3]:
def get_all_textgrid_files(path):
    filenames = []
    for root, dirs, files in os.walk(path):
        for file in files:
            if file.endswith(".TextGrid"):
                filenames.append(os.path.join(root, file))
    return filenames
word_level_timing_annotation = get_all_textgrid_files(word_level_timing)
print(word_level_timing_annotation)      

['./word_level_timing/r1A_wordlevel.TextGrid', './word_level_timing/r13A_wordlevel.TextGrid', './word_level_timing/r3A_wordlevel.TextGrid', './word_level_timing/r19B_wordlevel.TextGrid', './word_level_timing/r15B_wordlevel.TextGrid', './word_level_timing/r11B_wordlevel.TextGrid', './word_level_timing/r19A_wordlevel.TextGrid', './word_level_timing/r2B_wordlevel.TextGrid', './word_level_timing/r3B_wordlevel.TextGrid', './word_level_timing/r12B_wordlevel.TextGrid', './word_level_timing/r9A_wordlevel.TextGrid', './word_level_timing/r10A_wordlevel.TextGrid', './word_level_timing/r9B_wordlevel.TextGrid', './word_level_timing/r18A_wordlevel.TextGrid', './word_level_timing/r17B_wordlevel.TextGrid', './word_level_timing/r8B_wordlevel.TextGrid', './word_level_timing/r15A_wordlevel.TextGrid', './word_level_timing/r5A_wordlevel.TextGrid', './word_level_timing/r12A_wordlevel.TextGrid', './word_level_timing/r7B_wodlevel.TextGrid', './word_level_timing/r6B_wordleve.TextGrid', './word_level_timing/r5B

In [4]:
for index, files in enumerate(word_level_timing_annotation):
    if index == 1:
        break
    print(files)
    for index , tg in enumerate(textgrid.TextGrid.fromFile(files)):
        # print(tg) 
        for interval in tg:
            # print(interval.minTime, interval.maxTime, interval.mark)
            if len(interval.mark) > 1:
                doc = german_tagger(interval.mark)
                # print(doc)
                for token in doc:
                    tg.add_annotation(token.pos_)
                    # print(token.text, token.pos_, end=' ')
                    # print(interval.minTime, interval.maxTime)


./word_level_timing/r1A_wordlevel.TextGrid
Mhm PROPN 555.79196 556.1209
ja ADV 557.06442 557.21154
sehr ADV 562.64933 563.02646
richtig ADV 563.02646 563.41108
ja ADV 564.57219 564.75219
also ADV 565.86227 566.11227
mehr ADV 567.68027 568.3295
ja ADV 578.00879 578.35592
ja ADV 578.87442 579.12154
genau ADV 579.24662 579.53375
mittig ADV 585.1976 585.5676
ja ADV 585.98265 586.12133
ja ADV 587.58806 587.77806
direkt ADV 598.21737 598.4645
in ADP 598.4645 598.5345
der DET 598.5345 598.6745
Küche NOUN 598.6745 598.9245
steht VERB 598.9245 599.2821
achso VERB 601.4194 601.7394
ja ADV 604.6576 604.72473
ich PRON 604.72473 604.78473
glaub ADV 604.78473 605.06473
auch ADV 605.06473 605.42802
also ADV 605.44698 605.7441
allei- X 605.7441 606.0641
allein ADV 606.0641 606.3041
schon ADV 606.3041 606.6841
für ADP 606.6841 606.8841
die DET 606.9541 607.1041
Schuhe NOUN 607.1041 607.40329
ja ADV 609.19512 609.38225
ja ADV 617.25573 617.34285
ja ADV 617.34285 617.69285
genau ADV 617.69285 618.04344
j

### Need to define a dataset class first 

In [ ]:
class WordLevelTiming(torch.utils.data.Dataset):
    pass

In [ ]:
import torch
class RNNModel(torch.nn.Module):
    def __init__(self, hidden_dim, emdbedding_diminput_shape):
        super().__init__()
        self.hidden_dim = hidden_dim
        # self.word_embedding = 
        # self.lstm1 = nn.LSTM(embedding_dim, hidden_dim)
    def forward(self, x)
        # embeds = self.word_embedding
        # lstm_out, _ = self.lstm(embeds.view(len(x), 1, -1))
        # tag_space = self.hidden2tag(lstm_out.view(len(x), -1))
        # tag_scores = F.log_softmax(tag_space, dim=1)
        # return tag_scores

In [ ]:
import time
import torch
def train(train_loader,val_loader, model, loss_function, optimizer, epochs, device):
    print('train() called: model=%s, opt=%s(lr=%f), epochs=%d, device=%s\n' % \
          (type(model).__name__, type(optimizer).__name__,
           optimizer.param_groups[0]['lr'], epochs, device))

    history = {}
    history['loss'] = []
    history['val_loss'] = []
    history['acc'] = []
    history['val_acc'] = []
    start_time_sec = time.time()
    for epoch in range(epochs):
        model=model.to(device)
        model.train()
        train_loss = 0.0
        num_train_correct = 0
        num_train_examples = 0
        for i, batch in enumerate(train_loader):
            optimizer.zero_grad()
            
            images = batch['image'].to(device)
            target = batch['label'].to(device)

            output = model(images.float())
            loss = loss_function(output, target.float())
            
            loss.backward()
            optimizer.step()

            train_loss += loss.data.item()*images.size(0)
            num_train_correct  += (torch.max(output, 1)[1] == target).sum().item()
            num_train_examples += images.shape[0]
        train_acc =  num_train_correct / num_train_examples
        train_loss  = train_loss / len(train_loader.dataset)

        # --- EVALUATE ON VALIDATION SET -------------------------------------

        model.eval()
        val_loss       = 0.0
        num_val_correct  = 0
        num_val_examples = 0

        for batch in val_loader:
            images = batch['image'].to(device)
            target = batch['label'].to(device)

            output = model(images.float())
            loss = loss_function(output, target.float())

            val_loss         += loss.data.item()*images.size(0)
            num_val_correct  += (torch.max(output, 1)[1] == target).sum().item()
            num_val_examples += images.shape[0]
        val_acc  = num_val_correct / num_val_examples
        val_loss = val_loss / len(val_loader.dataset)
        print('Epoch %3d/%3d, train loss: %5.2f, train acc: %5.2f, val loss: %5.2f, val acc: %5.2f' % \
              (epoch+1, epochs, train_loss, train_acc, val_loss, val_acc))

        history['loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['acc'].append(train_acc)
        history['val_acc'].append(val_acc)

         # END OF TRAINING LOOP
    end_time_sec       = time.time()
    total_time_sec     = end_time_sec - start_time_sec
    time_per_epoch_sec = total_time_sec / epochs
    print()
    print('Time total:     %5.2f sec' % (total_time_sec))
    print('Time per epoch: %5.2f sec' % (time_per_epoch_sec))

    return history